In [1]:

import sys
import pathlib
import numpy as np
import pandas as pd
from rich.progress import track
from joblib import Parallel, delayed

import torch
from torch.utils.data import Dataset, DataLoader

sys.path.append("../src")
from utils.feature import FeatureStore
from utils.common import load_pickle, dump_pickle, save_cache

In [19]:
waypoint = load_pickle("../data/submit/test_waypoint.pkl", verbose=False)
bssid_map = load_pickle("../data/label_encode/map_bssid.pkl", verbose=False)

In [29]:
feature = load_pickle(f"../data/submit/path_data/00ff0c9a71cc37a2ebdd0f05.pkl", verbose=False)
wifi = feature.wifi.copy()

In [30]:
wifi["bssid"] = wifi["bssid"].map(bssid_map)

In [31]:
wifi

,timestamp,ssid,bssid,rssi,frequency,last_seen_timestamp
0,1180,3c1e7602176e050694e3a5cf8ba5f6f725e3ec51,126751.0,-58,2437,1573190310545
1,1180,5c072340f8e500f7e62819ab82bb8998ecd0ef4e,38681.0,-63,2462,1573190310849
2,1180,a4e38996343460efde1140975529e97c9f9aa60b,141828.0,-64,2412,1573190310225
3,1180,d0af9d9c2709796ee07a0432de0e26298a64e3e8,16170.0,-64,2412,1573190310235
4,1180,a4e38996343460efde1140975529e97c9f9aa60b,175953.0,-66,5745,1573190311283
...,...,...,...,...,...,...
2726,86716,da39a3ee5e6b4b0d3255bfef95601890afd80709,165195.0,-89,5745,1573190383326
2727,86716,5cf101239944d54a561d6d17580af348ee380169,217412.0,-89,5745,1573190391055
2728,86716,7b401ced936ac62a66fed1cc7ac81ec19c893e46,96588.0,-89,5745,1573190391013
2729,86716,41c0411b4b2ee7ed2aff925eaca9caf529d602de,140556.0,-89,5785,1573190385396


In [39]:
seq_len = 100
bssid = []
rssi = []
freq = []
min_idx = gdf.index.min()
max_idx = gdf.index.max()

for i, row in gdf.iterrows():
    ts_pre_wp = int(gdf.loc[i - 1, "timestamp"]) if i > min_idx else None
    ts_current_wp = int(gdf.loc[i, "timestamp"])
    ts_post_wp = int(gdf.loc[i + 1, "timestamp"]) if (i + 1) < max_idx else None

    _wifi = wifi.copy()
    # NOTE: ターゲットとなるwaypointとその前後のwaypointの間にあるデータを取得する。
    ts_wifi = _wifi["timestamp"].values
    pre_flag = (
        np.ones(len(ts_wifi)).astype(bool)
        if ts_pre_wp is None
        else (ts_pre_wp < ts_wifi)
    )
    psot_flag = (
        np.ones(len(ts_wifi)).astype(bool)
        if ts_post_wp is None
        else (ts_wifi < ts_post_wp)
    )
    _wifi = _wifi[pre_flag & psot_flag]

    _wifi = _wifi.sort_values(by="rssi", ascending=False)
    _wifi = _wifi.head(seq_len)

    _bssid = np.zeros(seq_len)
    _rssi = np.tile(-999, seq_len)
    _freq = np.tile(-999, seq_len)

    _bssid[: len(_wifi)] = _wifi["bssid"].fillna(0).to_numpy()
    _rssi[: len(_wifi)] = _wifi["rssi"].fillna(-999).to_numpy()
    _freq[: len(_wifi)] = _wifi["frequency"].fillna(-999).to_numpy()

    bssid.append(_bssid.astype("int32"))
    rssi.append(_rssi.astype("float32"))
    freq.append(_freq.astype("float32"))

In [40]:
bssid

[array([ 16170, 126751,  38681,  38681, 126751,  16170, 141828,  68836,
        175953,  68836, 141828, 175953, 217328,  42225,  42225, 205961,
        205961, 193015,  76439,  76439, 217328, 180641, 180641, 205474,
        205474, 162738, 186055, 186055,  40779, 165731,  40779, 165731,
        193015, 216665, 213541, 213541, 218995, 216665, 218995,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0], dtype=int32),
 array([ 16170,  16170, 126751,  16170,  16170,  16170,  38681,  38681,
         

In [9]:
for path_id, gdf in waypoint.groupby("path"):
    print(path_id)
    break
    seq_len = 100
    bssid = []
    rssi = []
    freq = []

    feature = load_pickle(f"../data/submit/path_data/{path_id}.pkl", verbose=False)
    wifi = feature.wifi.copy()
    wifi["bssid"] = wifi["bssid"].map(bssid_map)

    min_idx = gdf.index.min()
    max_idx = gdf.index.max()

    for i, row in gdf.iterrows():
        ts_pre_wp = int(gdf.loc[i - 1, "timestamp"]) if i > min_idx else None
        ts_current_wp = int(gdf.loc[i, "timestamp"])
        ts_post_wp = int(gdf.loc[i + 1, "timestamp"]) if (i + 1) < max_idx else None

        _wifi = wifi.copy()
        # NOTE: ターゲットとなるwaypointとその前後のwaypointの間にあるデータを取得する。
        ts_wifi = _wifi["timestamp"].values
        pre_flag = (
            np.ones(len(ts_wifi)).astype(bool)
            if ts_pre_wp is None
            else (ts_pre_wp < ts_wifi)
        )
        psot_flag = (
            np.ones(len(ts_wifi)).astype(bool)
            if ts_post_wp is None
            else (ts_wifi < ts_post_wp)
        )
        _wifi = _wifi[pre_flag & psot_flag]

        _wifi = _wifi.sort_values(by="rssi", ascending=False)
        _wifi = _wifi.head(seq_len)

        _bssid = np.zeros(seq_len)
        _rssi = np.tile(-999, seq_len)
        _freq = np.tile(-999, seq_len)

        display(_wifi.isnull().mean())

        _bssid[: len(_wifi)] = _wifi["bssid"].to_numpy()
        _rssi[: len(_wifi)] = _wifi["rssi"].to_numpy()
        _freq[: len(_wifi)] = _wifi["frequency"].to_numpy()

        bssid.append(_bssid.astype("int32"))
        rssi.append(_rssi.astype("float32"))
        freq.append(_freq.astype("float32"))

    break

00ff0c9a71cc37a2ebdd0f05
